In [251]:
%matplotlib inline

In [252]:
import ipywidgets
from IPython.display import display
from IPython.html import widgets

In [253]:
## Temp
values_container = dict()

In [257]:
def create_float_box(property_name,min=-1e15,max=1e15):
    float_box = widgets.BoundedFloatText(name=property_name,min=min, max=max, value=0, step=0.1)
    def handle_var_change(change):
        values_container[property_name] = change.new
    float_box.observe(handle_var_change, names="value")
    return float_box

def create_particle_type_box(property_name):
    particle_type_box = widgets.Dropdown(
        options=[None, "electrons", "ions", "electrons+ions"],
        value=None,
        disabled=False
    )
    def handle_particle_type_change(change):
        values_container[property_name] = change.new
    particle_type_box.observe(handle_particle_type_change, names="value")
    return particle_type_box

def create_text_box(property_name):
    text_box = widgets.Text(value="", placeholder="Enter ion type particle for e.g. p,He 2+", disabled=False)
    def handle_text_change(change):
        values_container[property_name] = change.new
    text_box.observe(handle_text_change, names="value")
    return text_box

In [258]:
grid = widgets.GridspecLayout(10, 2,width="700px")
## Row 0 - Magnetic Field
grid[0,0] = widgets.Label("B - Magnetic Field Magnitude(T):")
grid[0,1] = create_float_box("magnetic_field_magnitude")
## Row 1 - Density
grid[1,0] = widgets.Label("\u03C1 - Density(kg/m3):")
grid[1,1] = create_float_box("density_value",min=0)
## Row 2 - Particle Type
grid[2,0] = widgets.Label("Particle(Ion):")
grid[2,1] = create_text_box("particle_ion")
display(grid)

GridspecLayout(children=(Label(value='B - Magnetic Field Magnitude(T):', layout=Layout(grid_area='widget001'))…

In [259]:
print(values_container)

{}
